In [1]:
# import dependecies 

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

In [14]:
# read vaccinations csv

csv_file = "Resources/vaccinations.csv"
vaccinations_df = pd.read_csv(csv_file)
vaccinations_df.head()

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0


In [8]:
# set columns for first table

vaccination_rate_df = vaccinations_df[['location', 'iso_code', 'date', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'daily_vaccinations' ]].copy()
vaccination_rate_df.head()

,location,iso_code,date,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations
0,Afghanistan,AFG,2021-02-22,0.0,NaN,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,1367.0
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,1367.0
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,1367.0
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,1367.0


In [9]:
# set columns for second table

vaccination_daily_df = vaccinations_df[['location', 'iso_code', 'date', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred', 'daily_vaccinations_per_million' ]].copy()
vaccination_daily_df.head()

,location,iso_code,date,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million
0,Afghanistan,AFG,2021-02-22,0.0,NaN,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,34.0
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,34.0
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,34.0
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,34.0


In [16]:
# group by country for third table

# vaccination_country_df = vaccinations_df.groupby
# vaccination_country_df.head()

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43247,Zimbabwe,ZWE,2021-02-18,39.0,39.0,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN
43248,Zimbabwe,ZWE,2021-02-19,NaN,NaN,NaN,NaN,NaN,425.0,NaN,NaN,NaN,NaN,28.0
43249,Zimbabwe,ZWE,2021-02-20,NaN,NaN,NaN,NaN,NaN,425.0,NaN,NaN,NaN,NaN,28.0
43250,Zimbabwe,ZWE,2021-02-21,1314.0,1314.0,NaN,NaN,NaN,425.0,0.01,0.01,NaN,NaN,28.0


In [ ]:
# connect to SQL database 

protocol = 'postgresql'
username = '<user name>'
password = '<password>'
host = 'localhost'
port = 5432
database_name = ''
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

In [ ]:
# check tables

insp.get_table_names()

In [ ]:
# push first table to database  

vaccination_rate_df.to_sql(name='', con=engine, if_exists='append', index=False)

In [ ]:
# push second table to database  

vaccination_daily_df.to_sql(name='', con=engine, if_exists='append', index=False)

In [ ]:
# push thrid table to database  

# vaccination_country_df.to_sql(name='', con=engine, if_exists='append', index=False)

In [ ]:
# check data upload first table

pd.read_sql_query('select * from ', con=engine).head()

In [ ]:
# check data upload second table

pd.read_sql_query('select * from ', con=engine).head()

In [ ]:
# check data upload third table

pd.read_sql_query('select * from ', con=engine).head()